In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import warnings
import gc
import time

time_start=time.time()

In [2]:
train=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
test=pd.read_csv('invite_info_evaluate_1.txt',sep='\s+',names=['qid','uid','time'])

In [3]:
import numpy
import random
import scipy.special as special
import math
from math import log

class HyperParam(object):
    def __init__(self, alpha, beta): # 先初始化alpha和beta
        self.alpha = alpha
        self.beta = beta

    def sample_from_beta(self, alpha, beta, num, imp_upperbound):
        sample = numpy.random.beta(alpha, beta, num)
        I = []
        C = []
        for click_ratio in sample:
            imp = random.random() * imp_upperbound
            #imp = imp_upperbound
            click = imp * click_ratio
            I.append(imp)
            C.append(click)
        return I, C

    # 更新方式1
    def update_from_data_by_FPI(self, tries, success, iter_num, epsilon):
        '''estimate alpha, beta using fixed point iteration(类似EM估计)
        tries: 展示次数
        success: 点击次数
        iter_num: 迭代次数
        epsilon: 精度
        '''
        for i in range(iter_num):
            new_alpha, new_beta = self.__fixed_point_iteration(tries, success, self.alpha, self.beta)
            # 当迭代稳定时，停止迭代
            if abs(new_alpha - self.alpha) < epsilon and abs(new_beta - self.beta) < epsilon:
                break
            self.alpha = new_alpha
            self.beta = new_beta

    def __fixed_point_iteration(self, tries, success, alpha, beta):
        '''fixed point iteration'''
        sumfenzialpha = 0.0
        sumfenzibeta = 0.0
        sumfenmu = 0.0
        for i in range(len(tries)):
            sumfenzialpha += (special.digamma(success[i]+alpha) - special.digamma(alpha))
            sumfenzibeta += (special.digamma(tries[i]-success[i]+beta) - special.digamma(beta))
            sumfenmu += (special.digamma(tries[i]+alpha+beta) - special.digamma(alpha+beta))

        return alpha*(sumfenzialpha/sumfenmu), beta*(sumfenzibeta/sumfenmu)

    # 更新方式1
    def update_from_data_by_moment(self, tries, success):
        '''estimate alpha, beta using moment estimation(矩估计)
        tries: 展示次数
        success: 点击次数
        '''
        # 样本均值和样本方差
        mean, var = self.__compute_moment(tries, success)
        #print 'mean and variance: ', mean, var
        #self.alpha = mean*(mean*(1-mean)/(var+0.000001)-1)
        self.alpha = (mean+0.000001) * ((mean+0.000001) * (1.000001 - mean) / (var+0.000001) - 1)
        #self.beta = (1-mean)*(mean*(1-mean)/(var+0.000001)-1)
        self.beta = (1.000001 - mean) * ((mean+0.000001) * (1.000001 - mean) / (var+0.000001) - 1)

    def __compute_moment(self, tries, success):
        '''moment estimation(求样本均值和样本方差)'''
        ctr_list = []
        var = 0.0
        for i in range(len(tries)):
            ctr_list.append(float(success[i])/tries[i])
        mean = sum(ctr_list)/len(ctr_list)
        for ctr in ctr_list:
            var += pow(ctr-mean, 2)

        return mean, var/(len(ctr_list)-1)

In [4]:
test['target']=-1
data=pd.concat([train,test])
data=data.reset_index(drop=True)
from tqdm import tqdm
tqdm.pandas()
data['day']=data['time'].progress_apply(lambda x:int(x[1:].split('-H')[0]))

100%|██████████| 10630845/10630845 [00:20<00:00, 520031.82it/s]


In [5]:
data['fold']=0
data.loc[(data.day>=3840)&(data.day<=3846),'fold']=1
data.loc[(data.day>=3847)&(data.day<=3855),'fold']=2
data.loc[(data.day>=3856)&(data.day<=3860),'fold']=3
data.loc[(data.day>=3861)&(data.day<=3867),'fold']=4
data.loc[data.target==-1,'fold']=5

In [ ]:
###
##
#

In [3]:
member_info=pd.read_csv('member_info.txt',sep='\s+',names=['uid','sex','key_word','num_level','hot_level','regis_type','regis_platform',
                                                          'look_freq','a','b','c','d','e','A','B','C','D','E','salt','l_topic','topic_n'])

In [4]:
member_info=member_info[['uid','look_freq','A','B','C','D',]]

In [10]:
data=data.merge(member_info,on='uid',how='left')

In [11]:
data.head()

,index,qid,uid,time,target,day,fold,look_freq,A,B,C,D
0,0,Q2166419046,M401693808,D3865-H22,0,3865,4,weekly,MD825760,BR641329,PV587951,CT587951
1,1,Q1550017551,M3392373099,D3844-H11,0,3844,1,monthly,MD470265,BR470265,PV596305,CT470265
2,2,Q604029601,M2317670257,D3862-H15,0,3862,4,weekly,MD590290,BR800099,PV002320,CT314932
3,3,Q2350061229,M1618461867,D3849-H11,0,3849,2,daily,MD470265,BR470265,PV587951,CT587951
4,4,Q2443223942,M3544409350,D3867-H4,0,3867,4,monthly,MD224131,BR788740,PV426364,CT513159


In [25]:
encode_list=['qid','uid','A','B','C','D','look_freq']
for i in tqdm(encode_list):
    data[i+'_pro']=np.nan
    #break
    for fold in range(1,6):
        count=data[data.fold<fold].groupby(i)['target'].agg('count')
        sum_=data[data.fold<fold].groupby(i)['target'].agg('sum')
        HP = HyperParam(1, 1)
        HP.update_from_data_by_moment(count.values,sum_.values) 
        ctr=(sum_+ HP.alpha) / (count+ HP.alpha + HP.beta)
        data.loc[data.fold==fold ,i+'_pro']=data[data.fold==fold][i].map(ctr)
        data.loc[data.fold==fold ,i+'_pro']=data.loc[data.fold==fold ,i+'_pro'].fillna(HP.alpha / (HP.alpha + HP.beta))
        print(HP.alpha / (HP.alpha + HP.beta))



  0%|          | 0/7 [00:00<?, ?it/s]

0.24610192951574528
0.23720049152604264
0.23424362591617984
0.23820420655289806




 14%|█▍        | 1/7 [01:01<06:07, 61.25s/it]

0.24185488964065213
0.17969634834299322
0.17631040971727288
0.17716163637564009
0.17462245505454044




 29%|██▊       | 2/7 [02:31<05:50, 70.05s/it]

...

0.15329394891527084
0.1752496191944366
0.16805443929854383
0.15201890184024008
0.14911831090145813




 86%|████████▌ | 6/7 [04:00<00:33, 33.81s/it]

0.14206146570888067
0.22632710392133687
0.2146577887419691
0.24734979240116065
0.24701819085811713




100%|██████████| 7/7 [04:23<00:00, 37.58s/it]

0.2343314705264628


In [27]:
data.columns

Index(['index', 'qid', 'uid', 'time', 'target', 'day', 'fold', 'look_freq',
       'A', 'B', 'C', 'D', 'qid_pro', 'uid_pro', 'A_pro', 'B_pro', 'C_pro',
       'D_pro', 'look_freq_pro'],
      dtype='object')

In [28]:
data[['qid_pro', 'uid_pro', 'A_pro', 'B_pro', 'C_pro',
       'D_pro', 'look_freq_pro']].to_hdf('feature_uid_ctr_new.h5', key='data')